Using Paligemma with transformers

Paligemma is a new vision language model released by Google.In this notebook we will see how to use transformer for Paligemma interface.First, install below libraries with update flag as we need to use the latest version of transformers along with others

In [1]:
%pip install -q -U accelerate bitsandbytes git+https://github.com/huggingface/transformers.git 

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 5.0.0.dev0 which is incompatible.


In [15]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
import torch
import numpy as np
from PIL import Image
import requests

input_text="What is the color of bee standing on the flower?"
img_url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/bee.JPG?download=true"
imput_image = Image.open(requests.get(img_url, stream=True).raw)

In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModelForVision2Seq

processor = AutoProcessor.from_pretrained("google/paligemma-3b-pt-224")
model = AutoModelForVision2Seq.from_pretrained("google/paligemma-3b-pt-224")
from transformers import AutoTokenizer, PaliGemmaForConditionalGeneration, PaliGemmaProcessor
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id = "google/paligemma-3b-mix-224"
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.bfloat16)
processor = PaliGemmaProcessor.from_pretrained(model_id)